# Packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import pandas as pd
import pickle
import tensorflow as tf
import seaborn as sns
import scipy as sp

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, TimeSeriesSplit, train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from tsfresh.feature_selection.selection import select_features

from tsfresh.utilities.dataframe_functions import roll_time_series
from tsfresh import extract_features, select_features,extract_relevant_features
from tsfresh.utilities.dataframe_functions import impute,make_forecasting_frame
from tsfresh.feature_extraction import settings, EfficientFCParameters,MinimalFCParameters 

from tqdm import tqdm

In [3]:
from utils.model_summary_functions import feature_importance, metrics, summary_plot,multivariate_importance
from utils.plotting import plot_autocorrelation

In [4]:
import warnings # Serve para ignorar mensagens de alerta que aparecem 
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
plt.style.use('default')

plt.rcParams.update({
    'font.size': 16,
    'axes.linewidth': 2,
    'axes.titlesize': 20,
    'axes.edgecolor': 'black',
    'axes.labelsize': 20,
    'axes.grid': True,
    'lines.linewidth': 1.5,
    'lines.markersize': 6,
    'figure.figsize': (15, 6),
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'font.family': 'Arial',
    'legend.fontsize': 13,
    'legend.framealpha': 1,
    'legend.edgecolor': 'black',
    'legend.shadow': False,
    'legend.fancybox': True,
    'legend.frameon': True,
    'text.color': 'black',
    'axes.labelcolor': 'black',
})


In [6]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Importing the reduced dataset

In [7]:
df = pd.read_csv('Processed_datasets/reduced_dataset.csv.zip')
df

,0,1,2,3,4,5,6,7,8,9,...,83,84,85,86,87,88,89,90,91,92
0,5.472871,1.217982,6.230086,-0.950462,0.863161,-1.352218,4.725135,-4.429206,1.363932,-2.402084,...,0.042324,-0.628976,0.303736,1.050924,-0.319772,0.452098,-0.454401,-0.395371,-0.434333,-0.886681
1,5.456736,1.030217,6.275587,-0.838791,0.975329,-1.566040,4.830630,-4.338888,1.385022,-2.258257,...,0.149707,-0.568629,0.224101,0.962598,-0.370399,0.411446,-0.487105,-0.412622,-0.438104,-0.947290
2,5.456897,0.833749,6.308572,-0.730643,1.086719,-1.799117,4.927997,-4.225563,1.400620,-2.127380,...,0.244525,-0.515302,0.145613,0.862979,-0.416856,0.365906,-0.515017,-0.417493,-0.444786,-1.012555
3,5.472629,0.629589,6.327787,-0.628478,1.196396,-2.050700,5.015677,-4.089995,1.407393,-2.009958,...,0.322812,-0.469722,0.069574,0.750927,-0.458162,0.317323,-0.536380,-0.407113,-0.455174,-1.079685
4,5.501610,0.419481,6.331751,-0.534753,1.303558,-2.319721,5.092144,-3.933463,1.401821,-1.906113,...,0.380772,-0.431966,-0.002469,0.625161,-0.493436,0.267659,-0.549715,-0.378990,-0.469812,-1.145273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,-4.149586,1.311779,-1.733645,2.995828,-9.173312,2.279188,-0.460554,2.574104,1.066336,-3.507364,...,-0.038440,-0.516034,-0.068059,-1.804518,0.456283,0.363398,0.108329,-0.124854,-0.227933,0.457089
5996,-3.869611,1.194403,-1.635748,3.000731,-9.144402,2.275107,-0.266543,2.343572,1.135160,-3.420423,...,-0.124407,-0.503931,-0.087439,-1.899951,0.439299,0.353236,0.017841,-0.117174,-0.337113,0.505647
5997,-3.593313,1.100201,-1.565852,3.003578,-9.102322,2.267642,-0.078799,2.113092,1.207276,-3.346010,...,-0.207155,-0.505051,-0.082344,-1.931616,0.433744,0.329172,-0.068791,-0.108082,-0.390954,0.545995
5998,-3.312633,1.021631,-1.519267,3.001866,-9.045092,2.256996,0.095221,1.893864,1.292517,-3.293154,...,-0.283526,-0.518721,-0.052140,-1.902229,0.440545,0.296793,-0.149900,-0.107087,-0.395280,0.572288


In [8]:
eigenvector = 90
series = pd.DataFrame(df[df.columns[eigenvector]])  # Grabbing one of the 92 eigenvectors
series

,90
0,-0.395371
1,-0.412622
2,-0.417493
3,-0.407113
4,-0.378990
...,...
5995,-0.124854
5996,-0.117174
5997,-0.108082
5998,-0.107087


# Feature extraction - Tsfresh

In [9]:
# With the following function we can select highly correlated features
# it will remove the first feature that is correlated with any other feature

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            # we are interested in absolute coeff value
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [10]:
def create_input_dataset(eigenvector,MEMORY=20,HORIZON=1):
     
    # Extract an eigenvector from the dataframe
    time_series = eigenvector[eigenvector.columns[0]]
        
    # Create input dataset and target variable (1 step ahead)
    df_shift, _ = make_forecasting_frame(time_series,
                                          kind='Eigenvector',
                                          max_timeshift=MEMORY,
                                          rolling_direction=1)

    
    print('Feature extraction started')
        
    # Feature extraction using tsfresh
    X = extract_features(df_shift,
                             column_id="id",
                             column_sort="time",
                             column_value="value",
                             impute_function=impute,
                             n_jobs=6,
                             show_warnings=False,
                             disable_progressbar=True)
        
    print('Feature extraction completed')
        
    # Remove constant features
    X = X.loc[:, X.apply(pd.Series.nunique) != 1] 
        
    # Remove redundant features using univariate correlation
    corr_features = correlation(X, 0.95) 
    X_filtered = X.drop(corr_features, axis=1)
    X_selected = X_filtered.reset_index(level=0, drop=True)
    
    # Create dataset - Multistep ahead forecasting instead of one step ahead
    y = time_series.shift(-HORIZON)
    y.dropna(inplace=True,axis=0)

    X = X_selected[X_selected.index.isin(y.index)]
    y = y[y.index.isin(X_selected.index)]
    
    y = np.ravel(y)
    print(X.shape)
    
    
    # Remove features from statistical sets
    # X_selected = select_features(X,y,n_jobs=6)
        
    # Multivariate correlation
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0,shuffle=False,test_size=0.15)
    # multi_select = multivariate_importance(X_train, X_test, y_train, y_test)
    # multi_select.plot(relative=False,topn=4)
    # plt.show()
    
    # Train model
    mdl = GradientBoostingRegressor(n_estimators=100, random_state=0)
    mdl.fit(X_train, y_train)
    yhat = mdl.predict(X_test)
    memory_list_before_fs.append(mean_squared_error(y_test, yhat))

    
    # Multivariate feature selection
    selector = SelectFromModel(estimator=mdl, threshold='6*mean')
    selector.fit(X_train, y_train)

    selected_feat = X_train.columns[(selector.get_support())]
    X_final = X[selected_feat]
    print(X_final.shape)
    
    X_train, X_test, y_train, y_test = train_test_split(X_final, y, random_state=0, shuffle=False, test_size=0.15)
    mdl.fit(X_train, y_train)
    yhat = mdl.predict(X_test)
    memory_list_after_fs.append(mean_squared_error(y_test, yhat))
    
    
    df_final = pd.concat([X_final, pd.DataFrame(y)], axis=1)
    df_final.dropna(inplace=True, axis=0)

    print('Feature selection completed')
    
        

    metrics(yhat,y_test)
    print('Function has finished')
        
    return X,y


In [11]:
# memory_list_before_fs = []
# memory_list_after_fs = []


# rabe = np.arange(2, 14, 1)
# for memory in tqdm(rabe):
#     X, y = create_input_dataset(series, MEMORY=memory,HORIZON=15)

In [12]:
# plt.plot(rabe,memory_list_before_fs,label='Before')
# plt.plot(rabe, memory_list_after_fs,label='After')
# plt.legend()

In [13]:
HORIZON = 16
MLE = 0.094
dt = 0.25

print('This dataset forecasts {} Lyapunov times into the future'.format(HORIZON*dt*MLE))

This dataset forecasts 0.376 Lyapunov times into the future


# Generating dataset for all time series

In [14]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh import extract_features
from sklearn.model_selection import cross_val_score

In [15]:
def create_dataset_for_each_eigenvector(df,MEMORY,HORIZON):
    
    eigenvectors = np.arange(0,93,1)
    
    for i in tqdm(eigenvectors):
        
        # Extract an eigenvector from the dataframe
        time_series = pd.DataFrame(df[df.columns[i]]) 
        
        # Create input dataset and target variable
        df_shift, _ = make_forecasting_frame(time_series[time_series.columns[0]],
                                          kind='Eigenvector',
                                             max_timeshift=MEMORY,
                                          rolling_direction=1)
        
        print('Feature extraction started')
        
        # Feature extraction using tsfresh
        X = extract_features(df_shift,
                             column_id="id",
                             column_sort="time",
                             column_value="value",
                             impute_function=impute,
                             n_jobs=6,
                             show_warnings=False,
                             disable_progressbar=True)
        
        print('Feature extraction completed')
        
        # Remove constant features
        X = X.loc[:, X.apply(pd.Series.nunique) != 1] 
        
        # Remove redundant features using univariate correlation
        corr_features = correlation(X, 0.95) 
        X_filtered = X.drop(corr_features, axis=1)
        X_filtered = X_filtered.reset_index(level=0, drop=True)
        
        # Create dataset - Multistep ahead forecasting instead of one step ahead
        y = time_series.shift(-HORIZON)
        y.dropna(inplace=True, axis=0)

        X = X_filtered[X_filtered.index.isin(y.index)]
        y = y[y.index.isin(X_filtered.index)]

        y = np.ravel(y)
        
        print(X.shape)
        
        # Remove features from statistical sets
        X_selected = select_features(X, y, n_jobs=6)

        # Training a baseline model
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, shuffle=False, test_size=0.2)
        extra_trees = GradientBoostingRegressor(n_estimators=100, random_state=0)
        extra_trees.fit(X_train, y_train)
        
        # Multivariate feature selection
        selector = SelectFromModel(estimator=extra_trees, threshold='8*mean')
        selector.fit(X_train, y_train)
        
        print('Feature selection completed')
        
        selected_feat = X_train.columns[(selector.get_support())]
        X_final = X[selected_feat]
        df_final = pd.concat([X_final, pd.DataFrame(y)], axis=1)
        df_final.dropna(inplace=True, axis=0)
        
        print(X_final.shape)
        
        # Dataset after feature selection
        #df_final = pd.concat([X_selected, pd.DataFrame(y)], axis=1)
        
        
        # Print results from cross validation
        # cv = TimeSeriesSplit(n_splits=3)
        # scores = cross_val_score(extra_trees, X_final, y, scoring='r2', cv=cv)
        # print('Eigenvector '+str(i), 'R2 score: ',str(scores.mean()), '+-', str(scores.std()))
        
        # Export dataset
        #df_final.to_csv('dataframe_time_series/df_eigenvector_{0}__r2_cv_score_{1}.csv'.format(i, round(scores.mean(),4)))
        df_final.to_csv('dataframe_time_series/df_eigenvector_{0}.csv'.format(i))


In [17]:
create_dataset_for_each_eigenvector(df,MEMORY=4,HORIZON=12)

Rolling: 100%|██████████| 30/30 [00:10<00:00,  2.90it/s]


Feature extraction started
Feature extraction completed
(5987, 83)


  1%|          | 1/93 [01:22<2:06:22, 82.41s/it]

Feature selection completed
(5987, 2)


Rolling: 100%|██████████| 30/30 [00:09<00:00,  3.23it/s]


...

Feature selection completed
(5987, 3)


Rolling: 100%|██████████| 30/30 [00:11<00:00,  2.55it/s]


Feature extraction started
Feature extraction completed
(5987, 87)


100%|██████████| 93/93 [2:45:14<00:00, 106.61s/it]

Feature selection completed
(5987, 2)


Now that we have save the dataframe for all eigenvectors, the next notebook will use envolve finding the best model and hyperparameter turning